In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from os import path

root_path = '/content/gdrive/Shareddrives/PTDL WEB/Bài tập + Đồ án/Đồ án/Code_Crawl_Data_Shoppe'
dataset_path = path.join(root_path, 'dataset')
val_path = path.join(dataset_path, 'val_annotater')

!ls "$dataset_path"
!ls "$val_path"

annotated  preprocessing  raw  review_1.xlsx  val_annotater
2700-annotated_annotations.json  val-annotated_annotations.json  val-reannotated_annotations.json


In [3]:
import json

with open(path.join(val_path, 'val-annotated_annotations.json'), 'r') as f:
    datas = json.load(f)['examples']

datas

[{'content': 'đúng với mô_tả kho xấu',
  'seen_by': [{'annotator': 'tuanna21411@st.uel.edu.vn', 'annotator_id': 3},
   {'annotator': 'uyennnp21411@st.uel.edu.vn', 'annotator_id': 1},
   {'annotator': 'anhnhn21411@st.uel.edu.vn', 'annotator_id': 5},
   {'annotator': 'nguyennt21411@st.uel.edu.vn', 'annotator_id': 4},
   {'annotator': 'tamta21411@st.uel.edu.vn', 'annotator_id': 6}],
  'comments': [],
  'metadata': {'name': 'Pijama tay dài cổ vuông bộ đồ ngủ nữ pijama tiểu thư dài tay mặc nhà thu đông CARY',
   'cmtid': 13281399038,
   'itemid': 10656336686,
   'comment': 'Đúng với mô tả:kho\n\nXấu',
   'rating_star': 1},
  'example_id': 'b728221d-c69b-4ef9-b6f2-8cde84f4cf86',
  'annotations': [],
  'classifications': [{'correct': None,
    'class_id': 'd0a24e6b-7320-40a0-8d81-7ff0fc868976',
    'reviewed': False,
    'classname': 'Negative',
    'example_id': 'b728221d-c69b-4ef9-b6f2-8cde84f4cf86',
    'classified_by': [{'annotator': 'tuanna21411@st.uel.edu.vn',
      'timestamp': '2023-1

In [4]:
annotater2id = {}

for data in datas:
  for classification in data['classifications']:
    for annotator in classification['classified_by']:
      annotater2id[annotator['annotator'].split("@")[0]] = annotator['annotator_id']

annotater2id

{'tuanna21411': 3,
 'uyennnp21411': 1,
 'anhnhn21411': 5,
 'tamta21411': 6,
 'nguyennt21411': 4}

In [5]:
import pandas as pd
from collections import Counter

annotater_data = {k: [] for k in annotater2id.keys()}
annotater_data.update({
    'text': [],
    'cmtid': []
})

for data in datas:

  temp_dict = {i: 'None' for i in annotater_data.keys()}
  temp_dict['text'] = data['content']
  temp_dict['cmtid'] = data['metadata']['cmtid']

  for classification in data['classifications']:
    label = classification['classname']

    for annotator in classification['classified_by']:
      name = annotator['annotator'].split("@")[0]

      temp_dict[name] = label

  for k, v in temp_dict.items():
    annotater_data[k].append(v)

drop_columns = []
for k, v in annotater_data.items():
  if Counter(v).get('None', 0) >= len(datas) * 0.1:
    drop_columns.append(k)
    continue

  print(k, Counter(v))

annotater_df = pd.DataFrame.from_dict(annotater_data)
annotater_df = annotater_df.drop(drop_columns, axis = 1)
annotater_df

tuanna21411 Counter({'Positive': 112, 'Negative': 99, 'Neutral': 88, 'None': 1})
uyennnp21411 Counter({'Negative': 116, 'Positive': 100, 'Neutral': 84})
anhnhn21411 Counter({'Negative': 131, 'Positive': 121, 'Neutral': 45, 'None': 3})
tamta21411 Counter({'Negative': 132, 'Positive': 98, 'Neutral': 70})
nguyennt21411 Counter({'Neutral': 110, 'Positive': 106, 'Negative': 84})
text Counter({'đúng với mô_tả kho xấu': 1, '24 không một đôi mẹ ơi': 1, 'đúng với mô_tả đặt áo_khoác mà giao cái qq gì vậy màu_sắc saiii chất_liệu không đúng': 1, 'giao thiếu hàng': 1, 'đúng với mô_tả đúng màu_sắc nâu chất_liệu vải vải không được đẹp lắm nhưng phù_hợp giá_cả hơi rộng nhưng bé nhà mình thích nên vẫn đánh_giá cho shop năm sao': 1, 'chất_liệu mỏng màu sắc đen đúng với mô_tả yét khá là mỏng nhưng với giá này thì cũng ok': 1, 'màu sắc đen chất mỏng': 1, 'màu_sắc đúng đúng với mô_tả không chất_liệu mỏng áo bị lỗi': 1, 'mỏng và bự to rộng không mang được hàng về nhanh hơn dự_kiến a': 1, 'he was also an adv

tuanna21411 uyennnp21411 anhnhn21411 tamta21411 nguyennt21411  \
0      Negative     Negative    Negative   Negative       Neutral   
1       Neutral      Neutral    Negative   Negative       Neutral   
2      Negative     Negative    Negative   Negative      Negative   
3       Neutral      Neutral    Negative   Negative       Neutral   
4      Positive      Neutral    Positive   Positive      Positive   
..          ...          ...         ...        ...           ...   
295    Positive     Positive    Positive   Positive      Positive   
296    Positive     Positive    Positive   Positive      Positive   
297    Positive     Positive    Positive   Positive      Positive   
298    Negative     Negative    Negative   Negative      Negative   
299    Positive     Positive    Positive   Positive      Positive   

                                                  text        cmtid  
0                               đúng với mô_tả kho xấu  13281399038  
1                               24 không một đôi mẹ ơi  12198892833  
2    đúng với mô_tả đặt áo_khoác mà giao cái qq gì ...  13288012286  
3                                      giao thiếu hàng  11695680345  
4    đúng với mô_tả đúng màu_sắc nâu chất_liệu vải ...  13032398614  
..                                                 ...          ...  
295  mình nhận hàng rồi nha cho shop năm sao luôn k...  12626247367  
296  màu_sắc xanh than đúng với mô_tả đúng chất_liệ...  12967427776  
297  vải tăm nha mặc được đẹp lắm luôn mặc đi chơi ...  12502489602  
298      chất vải xấu như áo cũ mua về không mặc dk ln  13345935353  
299  tè vải đến kiểu_dáng đẹp hú_hồn mh không nghĩ ...  12639849781  

[300 rows x 7 columns]

In [6]:
from statsmodels.stats.inter_rater import aggregate_raters, fleiss_kappa

fleiss_matrix, categories = aggregate_raters(annotater_df.drop(['text', 'cmtid'], axis = 1))
kappa = fleiss_kappa(fleiss_matrix)
kappa

0.6819902671886527

In [7]:
error_sentences = []
for index, row in annotater_df.iterrows():
  values = list(row.to_dict().values())[:len(annotater2id)]
  if len(set(values)) > 1:
    error_sentences.append(
          {'cmtid': row['cmtid'],
          'annotated': values,
           'text': row['text']}
        )

print(len(error_sentences))

error_df = {i: [] for i in error_sentences[0].keys()}
for i in error_sentences:
  for k, v in i.items():
    error_df[k].append(v)

error_df = pd.DataFrame.from_dict(error_df)
error_df

125


cmtid                                          annotated  \
0    13281399038  [Negative, Negative, Negative, Negative, Neutral]   
1    12198892833    [Neutral, Neutral, Negative, Negative, Neutral]   
2    11695680345    [Neutral, Neutral, Negative, Negative, Neutral]   
3    13032398614  [Positive, Neutral, Positive, Positive, Positive]   
4    10353479733   [Neutral, Negative, Negative, Negative, Neutral]   
..           ...                                                ...   
120  13003587990    [Positive, Neutral, Positive, Neutral, Neutral]   
121  13255835599   [Neutral, Negative, Negative, Negative, Neutral]   
122  13227307206  [Negative, Negative, Positive, Positive, Posit...   
123  12842734854   [Positive, Positive, Neutral, Neutral, Positive]   
124  13274120293   [Positive, Neutral, Neutral, Positive, Positive]   

                                                  text  
0                               đúng với mô_tả kho xấu  
1                               24 không một đôi mẹ ơi  
2                                      giao thiếu hàng  
3    đúng với mô_tả đúng màu_sắc nâu chất_liệu vải ...  
4                                màu sắc đen chất mỏng  
..                                                 ...  
120  chất_liệu ok đúng với mô_tả được màu_sắc đẹp t...  
121  màu_sắc dbdj đúng với mô_tả nsjsj chất_liệu mỏ...  
122  màu_sắc xanh chất_liệu len mềm chất khá ok 56 ...  
123  chất_liệu bò đúng với mô_tả 80 màu_sắc hơi khá...  
124  đúng với mô_tả yes màu_sắc vàng <emoji> happy ...  

[125 rows x 3 columns]

In [8]:
error_sentences = []
for index, row in annotater_df.iterrows():
  values = list(row.to_dict().values())[:len(annotater2id)]
  if len(set(values)) > 1:
    error_sentences.append(
          {'cmtid': row['cmtid'],
          'annotated': values,
           'text': row['text']}
        )

print(len(error_sentences))

error_df = {i: [] for i in error_sentences[0].keys()}
for i in error_sentences:
  for k, v in i.items():
    error_df[k].append(v)

error_df = pd.DataFrame.from_dict(error_df)
error_df

125


cmtid                                          annotated  \
0    13281399038  [Negative, Negative, Negative, Negative, Neutral]   
1    12198892833    [Neutral, Neutral, Negative, Negative, Neutral]   
2    11695680345    [Neutral, Neutral, Negative, Negative, Neutral]   
3    13032398614  [Positive, Neutral, Positive, Positive, Positive]   
4    10353479733   [Neutral, Negative, Negative, Negative, Neutral]   
..           ...                                                ...   
120  13003587990    [Positive, Neutral, Positive, Neutral, Neutral]   
121  13255835599   [Neutral, Negative, Negative, Negative, Neutral]   
122  13227307206  [Negative, Negative, Positive, Positive, Posit...   
123  12842734854   [Positive, Positive, Neutral, Neutral, Positive]   
124  13274120293   [Positive, Neutral, Neutral, Positive, Positive]   

                                                  text  
0                               đúng với mô_tả kho xấu  
1                               24 không một đôi mẹ ơi  
2                                      giao thiếu hàng  
3    đúng với mô_tả đúng màu_sắc nâu chất_liệu vải ...  
4                                màu sắc đen chất mỏng  
..                                                 ...  
120  chất_liệu ok đúng với mô_tả được màu_sắc đẹp t...  
121  màu_sắc dbdj đúng với mô_tả nsjsj chất_liệu mỏ...  
122  màu_sắc xanh chất_liệu len mềm chất khá ok 56 ...  
123  chất_liệu bò đúng với mô_tả 80 màu_sắc hơi khá...  
124  đúng với mô_tả yes màu_sắc vàng <emoji> happy ...  

[125 rows x 3 columns]

In [9]:
error_df.to_excel(path.join(dataset_path, 'review_1.xlsx'))

In [10]:
import json

with open(path.join(val_path, 'val-reannotated_annotations.json'), 'r') as f:
    datas_2 = json.load(f)['examples']

datas_2

[{'content': 'nói_chung là xấu vải áo xấu lắm',
  'seen_by': [{'annotator': 'tamta21411@st.uel.edu.vn', 'annotator_id': 6},
   {'annotator': 'anhnhn21411@st.uel.edu.vn', 'annotator_id': 5},
   {'annotator': 'uyennnp21411@st.uel.edu.vn', 'annotator_id': 1},
   {'annotator': 'nguyennt21411@st.uel.edu.vn', 'annotator_id': 4},
   {'annotator': 'tuanna21411@st.uel.edu.vn', 'annotator_id': 3}],
  'comments': [],
  'metadata': {'name': 'ÁO THUN/T-SHIRT UNISEX FORM ÂU RAGE OF THE SEA (ROTS STUDIO) "CODITIONAL OF ROTS STUDIO"',
   'cmtid': 12163393239,
   'itemid': 16694691238,
   'comment': 'nói chung là xấu\nvải áo xấu lắm',
   'rating_star': 1},
  'example_id': 'c877cada-036c-4af0-bc22-840f0625dea2',
  'annotations': [],
  'classifications': [{'correct': None,
    'class_id': 'd0a24e6b-7320-40a0-8d81-7ff0fc868976',
    'reviewed': False,
    'classname': 'Negative',
    'example_id': 'c877cada-036c-4af0-bc22-840f0625dea2',
    'classified_by': [{'annotator': 'uyennnp21411@st.uel.edu.vn',
   

In [11]:
annotater5id = {}

for data in datas_2:
  for classification in data['classifications']:
    for annotator in classification['classified_by']:
      annotater5id[annotator['annotator'].split("@")[0]] = annotator['annotator_id']

annotater5id

{'uyennnp21411': 1,
 'anhnhn21411': 5,
 'tamta21411': 6,
 'nguyennt21411': 4,
 'tuanna21411': 3}

In [12]:
annotater_data_2 = {k: [] for k in annotater5id.keys()}
annotater_data_2.update({
    'text': [],
    'cmtid': []
})

for data in datas_2:

  temp_dict = {i: 'None' for i in annotater_data_2.keys()}
  temp_dict['text'] = data['content']
  temp_dict['cmtid'] = data['metadata']['cmtid']

  for classification in data['classifications']:
    label = classification['classname']

    for annotator in classification['classified_by']:
      name = annotator['annotator'].split("@")[0]

      temp_dict[name] = label

  for k, v in temp_dict.items():
    annotater_data_2[k].append(v)

drop_columns_2 = []
for k, v in annotater_data_2.items():
  if Counter(v).get('None', 0) >= len(datas) * 0.1:
    drop_columns_2.append(k)
    continue

  print(k, Counter(v))

annotater_df_2 = pd.DataFrame.from_dict(annotater_data_2)
annotater_df_2 = annotater_df_2.drop(drop_columns_2, axis = 1)
annotater_df_2

uyennnp21411 Counter({'Positive': 114, 'Negative': 106, 'Neutral': 80})
anhnhn21411 Counter({'Positive': 117, 'Negative': 116, 'Neutral': 67})
tamta21411 Counter({'Negative': 113, 'Positive': 110, 'Neutral': 77})
nguyennt21411 Counter({'Positive': 114, 'Negative': 107, 'Neutral': 79})
tuanna21411 Counter({'Positive': 116, 'Negative': 111, 'Neutral': 72, 'None': 1})
text Counter({'nói_chung là xấu vải áo xấu lắm': 1, 'giao bé quá thất_vọng': 1, 'màu_sắc nâu đúng với mô_tả 10 10 chất_liệu nỉ bông áo vải không dày không mỏng mình thích mặc rộng nên chọn size lớn sợ rộng nhma vừa in m6 40 không qua mông nhé': 1, 'chất_liệu đẹp màu_sắc đúng màu đúng với mô_tả ngoài đời còn đẹp hơn trên hình': 1, 'màu sắc đen chất mỏng': 1, 'áo kiểu gì mà tay dài tay ngắn không hiểu shop làm kiểu gì luôn': 1, 'tặng vớ đã đi rồi': 1, 'màu_sắc tyuio chất_liệu tyui đúng với mô_tả rewas tyuioplkjhgf': 1, 'chất_liệu mỏng như lá lúa mặc có phần khó_chịu đúng với mô_tả giao_diện đúng khoảng 80 màu_sắc đặt màu be mà

uyennnp21411 anhnhn21411 tamta21411 nguyennt21411 tuanna21411  \
0       Negative    Negative   Negative      Negative    Negative   
1       Negative    Negative   Negative      Negative    Negative   
2        Neutral    Positive   Positive      Positive    Positive   
3       Positive    Positive   Positive      Positive    Positive   
4        Neutral    Negative   Negative      Negative    Negative   
..           ...         ...        ...           ...         ...   
295      Neutral     Neutral    Neutral       Neutral     Neutral   
296      Neutral     Neutral    Neutral       Neutral     Neutral   
297     Positive    Positive    Neutral      Positive    Positive   
298     Positive    Positive   Positive      Positive    Positive   
299     Positive    Positive   Positive      Positive    Positive   

                                                  text        cmtid  
0                      nói_chung là xấu vải áo xấu lắm  12163393239  
1                                giao bé quá thất_vọng  12860404226  
2    màu_sắc nâu đúng với mô_tả 10 10 chất_liệu nỉ ...  13284156815  
3    chất_liệu đẹp màu_sắc đúng màu đúng với mô_tả ...  11671891913  
4                                màu sắc đen chất mỏng  10353479733  
..                                                 ...          ...  
295  ship thân_thiện đóng_gói cẩn_thận đc tặng thêm...  12280051239  
296  giao hàng nhanh quần rất dài so với ng m55 nha...  12352726771  
297  đúng với mô_tả <emoji> thumbs up </emoji> màu_...  11529783187  
298  đúng với mô_tả yes màu_sắc vàng <emoji> happy ...  13274120293  
299  đúng với mô_tả đúng màu_sắc nâu áo đẹp dễ mặc ...  12564152237  

[300 rows x 7 columns]

In [13]:
fleiss_matrix_2, categories = aggregate_raters(annotater_df_2.drop(['text', 'cmtid'], axis = 1))
kappa_2 = fleiss_kappa(fleiss_matrix_2)
kappa_2

0.8629463886881026